In [42]:
import qiskit as qk
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import EstimatorV2, SamplerV2
import numpy as np
import math as m

from adiabatic_optimisation import AdiabaticOptimisationProblem

In [43]:
from queue import LifoQueue
from copy import deepcopy

def pauli_at(pauli: str, index: int, length: int):
    label = []
    for i in range(length):
        if (i + 1) == index:
            label.append(pauli)
        else:
            label.append("I")

    return "".join(label)

def enumerate_choices(n: int, r: int) -> list:
    choices = []
    stack = LifoQueue()
    stack.put([])

    while not stack.empty():
        item = stack.get()

        if len(item) == r:
            choices.append(item)
        else:
            start_idx = (item[-1] + 1) if len(item) > 0 else 0
            for i in range(start_idx, n):
                new_item = deepcopy(item)
                new_item.append(i)
                stack.put(new_item)

    return choices

def generate_clique_hamiltonian(graph_order: int, clique_size: int) -> qk.quantum_info.operators.SparsePauliOp:
    length = graph_order * (graph_order - 1) // 2
    vertex_choices = enumerate_choices(graph_order, clique_size)
    idx = lambda n,r,c: int(-0.5 * r**2 + (n - 0.5) * r + c - r)

    # print("Vertex Choices:", len(vertex_choices), vertex_choices)

    result = qk.quantum_info.operators.SparsePauliOp([pauli_at("I", 0, length)], [0.])
    for vc in vertex_choices:
        h_alpha = qk.quantum_info.operators.Pauli(pauli_at("I", 0, length))
        edge_choices = enumerate_choices(clique_size, 2)

        for ec in edge_choices:
            edge_idx = idx(graph_order, vc[ec[0]], vc[ec[1]])
            # print(graph_order, vc[ec[0]], vc[ec[1]], edge_idx)
            edge_pauli = qk.quantum_info.operators.SparsePauliOp([pauli_at("I", edge_idx, length), pauli_at("Z", edge_idx, length)], [0.5, -0.5])
            h_alpha = edge_pauli.dot(h_alpha)

        result += h_alpha

    return result

def generate_iset_hamiltonian(graph_order: int, iset_size: int) -> qk.quantum_info.operators.SparsePauliOp:
    length = graph_order * (graph_order - 1) // 2
    vertex_choices = enumerate_choices(graph_order, iset_size)
    idx = lambda n,r,c: int(-0.5 * r**2 + (n - 0.5) * r + c - r)

    # print("Vertex Choices:", len(vertex_choices), vertex_choices)

    result = qk.quantum_info.operators.SparsePauliOp([pauli_at("I", 0, length)], [0.])
    for vc in vertex_choices:
        h_alpha = qk.quantum_info.operators.Pauli(pauli_at("I", 0, length))
        edge_choices = enumerate_choices(iset_size, 2)

        for ec in edge_choices:
            edge_idx = idx(graph_order, vc[ec[0]], vc[ec[1]])
            # print(graph_order, vc[ec[0]], vc[ec[1]], edge_idx)
            edge_pauli = qk.quantum_info.operators.SparsePauliOp([pauli_at("I", edge_idx, length), pauli_at("Z", edge_idx, length)], [0.5, 0.5])
            h_alpha = edge_pauli.dot(h_alpha)

        result += h_alpha

    return result

def generate_ramsey_hamiltonian(graph_order: int, clique_size: int, iset_size: int) -> qk.quantum_info.operators.SparsePauliOp:
    clique_ham = generate_clique_hamiltonian(graph_order, clique_size)
    iset_ham = generate_iset_hamiltonian(graph_order, iset_size)

    # print(clique_ham.to_matrix().astype(np.float32))
    # print(iset_ham.to_matrix().astype(np.float32))
    # print((clique_ham + iset_ham).to_matrix().astype(np.float32))

    return clique_ham + iset_ham

In [44]:
def generate_initial_hamiltonian(graph_order: int) -> qk.quantum_info.operators.SparsePauliOp:
    length = graph_order * (graph_order - 1) // 2
    result = qk.quantum_info.operators.SparsePauliOp([pauli_at("I", 0, length)], [0.])
    for l in range(length):
        result += qk.quantum_info.operators.SparsePauliOp([pauli_at("I", l, length), pauli_at("X", l, length)], [0.5, -0.5])

    return result

## Tests

### R(2,5)

In [45]:
# test_clique = 2
# test_iset = 5
# test_sizes = [2, 3, 4, 5]
# test_problem_hamiltonians = []
# test_initial_hamiltonians = []
# for n in test_sizes:
#     test_problem_hamiltonians.append(generate_ramsey_hamiltonian(n, test_clique, test_iset))
#     test_initial_hamiltonians.append(generate_initial_hamiltonian(n))

### R(2,2)

In [46]:
# test_clique = 2
# test_iset = 2
# test_sizes = [2, 3, 4]
# test_problem_hamiltonians = []
# test_initial_hamiltonians = []
# for n in test_sizes:
#     test_problem_hamiltonians.append(generate_ramsey_hamiltonian(n, test_clique, test_iset))
#     test_initial_hamiltonians.append(generate_initial_hamiltonian(n))

### R(3,3)

In [ ]:
test_clique = 3
test_iset = 3
test_sizes = [5, 6]
test_problem_hamiltonians = []
test_initial_hamiltonians = []
for n in test_sizes:
    test_problem_hamiltonians.append(generate_ramsey_hamiltonian(n, test_clique, test_iset))
    test_initial_hamiltonians.append(generate_initial_hamiltonian(n))

### Execution

In [47]:
simulator = AerSimulator()
estimator = EstimatorV2()

def execute_circuit_estimator(aqo: AdiabaticOptimisationProblem, initial_state) -> np.ndarray:
    aqo.generate(initial_state)
    aqo.circuit = aqo.circuit.decompose()

    observables = [aqo.problem_hamiltonian]
    pub = (qk.transpile(aqo.circuit, simulator), observables)
    job = estimator.run([pub])

    return job.result()[0].data.evs[0]

In [48]:
simulator = AerSimulator()
sampler = SamplerV2()

def execute_circuit_sampler(aqo: AdiabaticOptimisationProblem, initial_state) -> np.ndarray:
    aqo.generate(initial_state)
    aqo.circuit.measure_all()
    aqo.circuit = aqo.circuit.decompose()

    job = sampler.run([qk.transpile(aqo.circuit, simulator)], shots=1000)
    return job.result()[0].data.meas.get_counts()

In [49]:
test_result_energy = []
test_result_states = []
for i in range(len(test_sizes)):
    aqop = AdiabaticOptimisationProblem(
        initial_hamiltonian=test_initial_hamiltonians[i],
        problem_hamiltonian=test_problem_hamiltonians[i],
        time=5.,
        steps=10
    )

    equal_prob = 1 / m.sqrt(2**aqop.num_qubits)
    test_result_energy.append(execute_circuit_estimator(aqop, [equal_prob for _ in range(2**aqop.num_qubits)]))
    test_result_states.append(execute_circuit_sampler(aqop, [equal_prob for _ in range(2**aqop.num_qubits)]))

In [50]:
print(f"{'':=<64}")
for i in range(len(test_sizes)):
    print(f"R({test_clique}, {test_iset}) ?= {test_sizes[i]}")
    print(f"{'':-<12}")

    eigvals = np.linalg.eigvals(test_problem_hamiltonians[i])
    groundstate_energy = np.min(eigvals)
    groundstate_energy_degeneracy = np.sum(eigvals == groundstate_energy)

    print("Classical numpy Groundstate energy:", groundstate_energy)
    print("Classical numpy Degeneracy:", groundstate_energy_degeneracy)
    print("AQO output energy:", test_result_energy[i])
    print("Approximate AQO output state:")
    prob_sum = 0.
    state_kets = []
    for key in test_result_states[i].keys():
        prob = test_result_states[i][key] / 1000
        prob_sum += prob
        state_kets.append(f"{m.sqrt(prob):.2f}|{key}>")
    print("\tProbability sum:", prob_sum)
    print(f"\t{' + '.join(state_kets)}")
    print(f"{'':=<64}")

R(2, 2) ?= 2
------------
Classical numpy Groundstate energy: (1+0j)
Classical numpy Degeneracy: 2
AQO output energy: 1.0000000000000002
Approximate AQO output state:
	Probability sum: 1.0
	0.72|1> + 0.70|0>
R(2, 2) ?= 3
------------
Classical numpy Groundstate energy: (3+0j)
Classical numpy Degeneracy: 8
AQO output energy: 3.000000000000001
Approximate AQO output state:
	Probability sum: 1.0
	0.35|011> + 0.33|010> + 0.36|001> + 0.35|100> + 0.38|101> + 0.35|000> + 0.35|111> + 0.36|110>
R(2, 2) ?= 4
------------
Classical numpy Groundstate energy: (6+0j)
Classical numpy Degeneracy: 64
AQO output energy: 6.000000000000336
Approximate AQO output state:
	Probability sum: 1.0000000000000007
	0.13|010110> + 0.13|001001> + 0.14|000111> + 0.14|001011> + 0.13|000001> + 0.13|000011> + 0.14|010000> + 0.11|111110> + 0.11|011101> + 0.10|101101> + 0.13|111100> + 0.13|010001> + 0.12|011111> + 0.15|111011> + 0.12|111000> + 0.14|101110> + 0.12|000101> + 0.12|010010> + 0.13|000110> + 0.12|110010> + 0.10